In [ ]:
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
from pathlib import Path
import math
import numpy as np
import torch
from PIL import Image
from nerfstudio.utils.eval_utils import eval_setup
from nerfstudio.cameras.cameras import Cameras
from nerfstudio.field_components.field_heads import FieldHeadNames

In [ ]:
config_path = "/root/yairshp/output_models/transformations/outputs/table/mipnerf/2023-09-27_212748/config.yml"
config_path = Path("/", *config_path.split("/"))
checkpoint_path = "/root/yairshp/output_models/transformations/outputs/table/mipnerf/2023-09-27_212748/nerfstudio_models"

fg_config_path = "/root/yairshp/output_models/transformations/outputs/ficus/mipnerf/2023-09-23_212151/config.yml"
fg_config_path = Path("/", *fg_config_path.split("/"))
fg_checkpoint_path = "/root/yairshp/output_models/transformations/outputs/ficus/mipnerf/2023-09-23_212151/nerfstudio_models",

In [ ]:
_, pipeline, _, _ = eval_setup(config_path, eval_num_rays_per_chunk=None, test_mode="inference", checkpoint_path=checkpoint_path)
_, fg_pipeline, _, _ = eval_setup(fg_config_path, eval_num_rays_per_chunk=None, test_mode="inference", checkpoint_path=fg_checkpoint_path)
pipeline.datamanager.setup_train()
fg_pipeline.datamanager.setup_train()

In [ ]:
current_index = torch.tensor(119)
camera_transforms = pipeline.datamanager.train_camera_optimizer(current_index.unsqueeze(dim=0))
current_camera = pipeline.datamanager.train_dataparser_outputs.cameras[current_index].to('cuda')
modified_camera = Cameras(
    fx=torch.tensor([[548.9938],[548.9938]]).to('cuda'),
    fy=torch.tensor([[548.9938],[548.9938]]).to('cuda'),
    cx=torch.tensor(256.).to('cuda'),
    cy=torch.tensor(256.).to('cuda'),
    camera_to_worlds=current_camera.camera_to_worlds,
    camera_type=current_camera.camera_type,
    times=current_camera.times
)
current_ray_bundle = modified_camera.generate_rays(torch.tensor(list(range(1))).unsqueeze(-1), camera_opt_to_camera=camera_transforms, aabb_box=None)
fg_ray_bundle = modified_camera.generate_rays(torch.tensor(list(range(1))).unsqueeze(-1), camera_opt_to_camera=camera_transforms, aabb_box=None)

In [ ]:
fg_camera_transformations = camera_transforms.clone()

fg_translation = fg_camera_transformations.clone()
fg_translation[0][3] = 0.
fg_translation[1][3] = -1.33
fg_translation[2][3] = 0.

fg_scale = fg_camera_transformations.clone()
fg_scale[0][0] = 1.1
fg_scale[1][1] = 1.1
fg_scale[2][2] = 1.1

fg_rotation_z = fg_camera_transformations.clone()
fg_rotation_z[0][0] = math.cos(math.pi / 4)
fg_rotation_z[1][1] = math.cos(math.pi / 4)
fg_rotation_z[0][1] = math.sin(math.pi / 4)
fg_rotation_z[1][0] = -math.sin(math.pi / 4)

In [ ]:
with torch.no_grad():
    outputs = pipeline.model.get_outputs_for_camera_ray_bundle(current_ray_bundle, fg_pipeline=fg_pipeline, fg_camera_ray_bundle=fg_ray_bundle)

In [ ]:
image = (outputs['rgb_coarse'].cpu().numpy() * 255).astype(np.uint8)
image = Image.fromarray(image)
image


In [ ]:
# num_rays_per_chunk = pipeline.model.config.eval_num_rays_per_chunk
# image_height, image_width = current_ray_bundle.origins.shape[:2]
# num_rays = len(current_ray_bundle)
# # outputs_lists = defaultdict(list)
# rgb_fine_outputs = []
# with torch.no_grad():
#     for i in range(0, num_rays, num_rays_per_chunk):
#         start_idx = i
#         end_idx = i + num_rays_per_chunk
#         ray_bundle = current_ray_bundle.get_row_major_sliced_ray_bundle(start_idx, end_idx)
#         if pipeline.model.collider is not None:
#             ray_bundle = pipeline.model.collider(ray_bundle)
#         if pipeline.model.field is None:
#                 raise ValueError("populate_fields() must be called before get_outputs")
#         ray_samples_uniform = pipeline.model.sampler_uniform(ray_bundle)
#         field_outputs_coarse = pipeline.model.field.forward(ray_samples_uniform)
#         weights_coarse = ray_samples_uniform.get_weights(field_outputs_coarse[FieldHeadNames.DENSITY])
#         rgb_coarse = pipeline.model.renderer_rgb(rgb=field_outputs_coarse[FieldHeadNames.RGB], weights=weights_coarse)
#         ray_samples_pdf = pipeline.model.sampler_pdf(ray_bundle, ray_samples_uniform, weights_coarse)
#         field_outputs_fine = pipeline.model.field.forward(ray_samples_pdf)
#         weights_fine = ray_samples_pdf.get_weights(field_outputs_fine[FieldHeadNames.DENSITY])
#         rgb_fine = pipeline.model.renderer_rgb(
#             rgb=field_outputs_fine[FieldHeadNames.RGB],
#             weights=weights_fine,
#         )
#         rgb_fine_outputs.append(rgb_fine)
#     rgb_fine = torch.cat(rgb_fine_outputs).view(image_height, image_width, -1)

In [ ]:
# image = (rgb_fine.cpu().numpy() * 255).astype(np.uint8)
# image = Image.fromarray(image)
# image